
## Prerequisites

gensim==3.6.0

In [123]:
import os

from ast import literal_eval

from string import punctuation

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from gensim.models import Word2Vec, KeyedVectors
from gensim.models import FastText

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split


lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words('english'))

In [126]:
import numexpr as ne

ne.set_num_threads(ne.detect_number_of_cores())

2

In [2]:
df = pd.read_csv("../jigsaw-toxic-comment-classification-challenge/train.csv")

In [3]:
def preprocess_text(tokenizer, lemmatizer, stop_words, punctuation, text): 
    tokens = tokenizer(text.lower())
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    return [token for token in lemmas if token not in stop_words and token not in punctuation]

In [4]:
bool_load = True

if not bool_load:
    df['cleaned'] = df.comment_text.apply(lambda x: preprocess_text(word_tokenize, lemmatizer, stop_words, punctuation, x))

In [5]:
bool_save = False

if bool_save:
    df.to_csv("../jigsaw-toxic-comment-classification-challenge/train.csv")

In [6]:
df_sample = df.sample(100000)

In [7]:
df_sample.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned
24039,24039,3f7df081f41a7ea3,"Also, the preview is free.",0,0,0,0,0,0,"['also', 'preview', 'free']"
107340,107340,3dccfe67cbc88d80,"Irrelevancy, unconnectedness, competition to b...",0,0,0,0,0,0,"['irrelevancy', 'unconnectedness', 'competitio..."
87489,87489,ea0d65e2c36e8d21,How am I supposed to test how redirecting work...,0,0,0,0,0,0,"['supposed', 'test', 'redirecting', 'work', 's..."
159053,159053,f7be51022c3e60bf,How rude \n\nSeriously? My sillypeppymacspeed ...,1,0,1,0,0,0,"['rude', 'seriously', 'sillypeppymacspeed', 'p..."
106724,106724,3acf61c2d1ec75ec,"""\nThat 'Wada's Memory' is lovely. Most around...",0,0,0,0,0,0,"['``', ""'wada"", ""'s"", 'memory', 'lovely', 'aro..."


### Train the model from scratch

Train our first model based on the vocabulary from df_sample: 

In [8]:
# With initialization model trained for 5 epochs 

df_sample_cleaned_list = [literal_eval(s) for s in df_sample.cleaned.tolist()]

model = Word2Vec(sentences=df_sample_cleaned_list, 
         size=100,      # embedding vector size
         min_count=5,   # consider words that occured at least 5 times
         window=5)

In [9]:
# Continue training the model 

model.train(sentences=df_sample_cleaned_list, 
            total_examples=model.corpus_count,
            epochs=30
           )

(100574369, 118974570)

In [48]:
#model.wv.vocab # to look at vocabulary

In [11]:
model.wv.most_similar('people')

[('thing', 0.6192317008972168),
 ('others', 0.6143143177032471),
 ('person', 0.581950843334198),
 ('everyone', 0.5551775693893433),
 ('editor', 0.5309818983078003),
 ('personally', 0.527014970779419),
 ('guy', 0.5265326499938965),
 ('anyone', 0.5155367255210876),
 ('way', 0.5107394456863403),
 ('individual', 0.5079406499862671)]

### The next approach is to try to use the already pretrained model, which can be downloaded from here:

https://github.com/RaRe-Technologies/gensim-data

model:   
GoogleNews-vectors-negative300.bin

In [12]:
#os.getcwd()

In [13]:
model = KeyedVectors.load_word2vec_format(
    os.getcwd() + os.sep + "GoogleNews-vectors-negative300.bin", binary=True
)

In [14]:
# You can try to use GloVe model too and experiment with it: <- later
# import gensim.downloader as api
# model = api.load('glove-wiki-gigaword-100')

## Words distance 

# 1 - Cosine similarity

To measure how similar two words are, we need a way to measure the degree of similarity between two embedding vectors for the two words. Given two vectors $u$ and $v$, cosine similarity is defined as follows: 

$$\text{CosineSimilarity(u, v)} = \frac {u . v} {||u||_2 ||v||_2} = cos(\theta) \tag{1}$$

where $u.v$ is the dot product (or inner product) of two vectors, $||u||_2$ is the norm (or length) of the vector $u$, and $\theta$ is the angle between $u$ and $v$. This similarity depends on the angle between $u$ and $v$. If $u$ and $v$ are very similar, their cosine similarity will be close to 1; if they are dissimilar, the cosine similarity will take a smaller value. 

<img src="cosine_sim.png" style="width:800px;height:250px;">
<caption><center> **Figure 1**: The cosine of the angle between two vectors is a measure of how similar they are</center></caption>

**Exercise**: Implement the function `cosine_similarity()` to evaluate similarity between word vectors.

**Reminder**: The norm of $u$ is defined as $ ||u||_2 = \sqrt{\sum_{i=1}^{n} u_i^2}$

In [15]:
def cosine_similarity(w1, w2):
    """
    Cosine similarity between w1 and w2
    
    Arguments:
        w1 : word vector        
        w2 : word vector 
    Returns:
        cosine_similarity 
    """
    if (not np.any(w1) or not np.any(w2)): # check input is not zero-vector
        return 0
    
    # Dot product between w1 and w2
    dot = np.dot(w1, w2)
    # L2 norm of w1
    norm_u = np.linalg.norm(w1) 
    # L2 norm of w2 
    norm_v = np.linalg.norm(w2) 
    # Cosine similarity 
    cosine_similarity = dot / (norm_u * norm_v)
    
    return cosine_similarity

In [16]:
father = model.get_vector("father")
mother = model.get_vector("mother")

ball = model.get_vector("ball")
crocodile = model.get_vector("crocodile")

france = model.get_vector("france")
paris = model.get_vector("paris")
italy = model.get_vector("italy")
rome = model.get_vector("rome")

kiev = model.get_vector("kiev")
ukraine = model.get_vector("ukraine")

In [17]:
fast_print = lambda u, v, tag1, tag2: print(
    "cosine_similarity({t1}, {t2}) = ".format(t1 = tag1, t2 = tag2), cosine_similarity(u, v)
)

fast_print(father, mother, "father", "mother")
fast_print(ball, crocodile, "ball", "crocodile")
fast_print(france - paris, rome - italy, "france - paris", "rome - italy")
fast_print(kiev, ukraine, "kiev", "ukraine")

cosine_similarity(father, mother) =  0.79014826
cosine_similarity(ball, crocodile) =  0.10283584
cosine_similarity(france - paris, rome - italy) =  -0.1988747
cosine_similarity(kiev, ukraine) =  0.3738725


**Approximate expected output**:

<table>
    <tr>
        <td>
            **cosine_similarity(father, mother)** =
        </td>
        <td>
         0.79014826
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(ball, crocodile)** =
        </td>
        <td>
         0.10283585
        </td>
    </tr>
        <tr>
        <td>
            **cosine_similarity(france - paris, rome - italy)** =
        </td>
        <td>
         -0.421037
        </td>
    </tr>
</table>

## 2 - Word analogy task

In the word analogy task, we complete the sentence <font color='brown'>"*a* is to *b* as *c* is to **____**"</font>. An example is <font color='brown'> '*man* is to *woman* as *king* is to *queen*' </font>. In detail, we are trying to find a word *d*, such that the associated word vectors $e_a, e_b, e_c, e_d$ are related in the following manner: $e_b - e_a \approx e_d - e_c$. We will measure the similarity between $e_b - e_a$ and $e_d - e_c$ using cosine similarity. 

**Exercise**: Complete the code below to be able to perform word analogies!

***Note***: here you will need to complete a function in the sections, which are marked as:

```
# ----- Start ----- #
Your code should be written in-between the lines
# ------ End ------ #
```


In [18]:
def find_word_analogy(word_1, word_2, word_3, model):
    """
    Finds the word to complete analogy (see explanation above): a is to b as c is to ____. 
    
    Arguments:
    word_1 -- a word, string
    word_2 -- a word, string
    word_3 -- a word, string
    model -- word embeddings model 
    
    Returns:
    best_word --  the word such that v_1 - v_2 is close to v_best_word - v_3, as measured by cosine similarity
    """
    # convert words to lower case
    word_1, word_2, word_3 = word_1.lower(), word_2.lower(), word_3.lower()
    
    # ----- Start ----- #
    # Get the word embeddings v_a, v_b and v_c (≈1-3 lines)
    fast_get = lambda word: model.get_vector(word)
    e_1, e_2, e_3 = tuple(map(fast_get, [word_1, word_2, word_3]))
    # ------ End ------ #
    
    words = list(model.vocab.keys())
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_word = None                   # Initialize best_word with None

    # Loop over the whole word vector set
    for w in words:        
        e_j = fast_get(w)
        # to avoid best_word being one of the input words, skip them and continue iteration.
        if w in [word_1, word_2, word_3]:
            continue
        
        # ----- Start ----- #
        # Compute cosine similarity between the vector (e_2 - e_1) and the vector ((w's vector) - e_3)
        cosine_sim = cosine_similarity(e_2 - e_1, e_j - e_3)
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
        # do not forget to set new max_cosine_sim to the current value and best_word as well
        if cosine_sim > max_cosine_sim:
            max_cosine_sim = cosine_sim
            best_word = w
        # ------ End ------ #
        
    return best_word

In [19]:
triads_to_try = [
    ('man', 'woman', 'king'), 
    ('bad', 'good', 'sad'), 
    ('man', 'woman', 'boy'), 
    ('small', 'smaller', 'large')
]

for triad in triads_to_try:
    print('{} -> {} :: {} -> {}'.format(*triad, find_word_analogy(*triad, model)))

man -> woman :: king -> queen
bad -> good :: sad -> wonderful
man -> woman :: boy -> girl
small -> smaller :: large -> larger


**Expected Output**:

<table>
    <tr>
        <td>
            **man -> woman** ::
        </td>
        <td>
         king -> queen
        </td>
    </tr>
        <tr>
        <td>
            **bad -> good** ::
        </td>
        <td>
         sad -> wonderful
        </td>
    </tr>
        <tr>
        <td>
            **man -> woman ** ::
        </td>
        <td>
         boy -> girl
        </td>
    </tr>
        <tr>
        <td>
            **small -> smaller ** ::
        </td>
        <td>
         large -> larger
        </td>
    </tr>
</table>

#### The next part of the task is to:  

1. Train your own W2V model using the proposed method above. Use all of the tokens created after your preprocessing pipeline in the previous tasks. (deleting stop_words, punctuation, lowercasing, etc - play as you want).  
2. Use obtained vectors to obtain text vectors using such pipeline: 
  1. For each word in a preprocessed text, get a word vector from the W2V model. 
  2. Add them together to obtain vectors for texts (sum them together, or get mean vector) 
3. Use obtained text vectors as a text representation to perform a text classification task.  
   Proposed - use binary classification (for example: select only 'obscene' text and clean and try to distinguish them one from another)
4. Calculate the metrics - TP, FP, FN, TN, precision, recall, F1 score, F2 score, accurary. 


In [20]:
from gensim.models.callbacks import CallbackAny2Vec


class callback_custom(CallbackAny2Vec):
    def __init__(self):
         self.epoch = 0

    def on_epoch_end(self, model):
        print("Iteration {:3}".format(self.epoch+1))
        self.epoch += 1

In [21]:
# init w2v model
n_dimensions = 300

model_w2v = Word2Vec(sentences=df_sample_cleaned_list, 
                     size=n_dimensions, min_count=5, window=5,
                     callbacks=[callback_custom()]
                    )

Iteration   1
Iteration   2
Iteration   3
Iteration   4
Iteration   5


In [22]:
# model training
number_of_iterations = 50

model_w2v.train(sentences=df_sample_cleaned_list, 
            total_examples=model_w2v.corpus_count,
            epochs=number_of_iterations
           )

Iteration   6
Iteration   7
Iteration   8
Iteration   9
Iteration  10
Iteration  11
Iteration  12
Iteration  13
Iteration  14
Iteration  15
Iteration  16
Iteration  17
Iteration  18
Iteration  19
Iteration  20
Iteration  21
Iteration  22
Iteration  23
Iteration  24
Iteration  25
Iteration  26
Iteration  27
Iteration  28
Iteration  29
Iteration  30
Iteration  31
Iteration  32
Iteration  33
Iteration  34
Iteration  35
Iteration  36
Iteration  37
Iteration  38
Iteration  39
Iteration  40
Iteration  41
Iteration  42
Iteration  43
Iteration  44
Iteration  45
Iteration  46
Iteration  47
Iteration  48
Iteration  49
Iteration  50
Iteration  51
Iteration  52
Iteration  53
Iteration  54
Iteration  55


(167618003, 198290950)

In [23]:
#model_w2v.wv.vocab

In [24]:
model_w2v.wv.most_similar('people')

[('others', 0.5339361429214478),
 ('thing', 0.5025500059127808),
 ('way', 0.4797881543636322),
 ('person', 0.4717697203159332),
 ("n't", 0.4668528437614441),
 ('would', 0.4465913474559784),
 ("'re", 0.4391423165798187),
 ('editor', 0.4381656348705292),
 ('everyone', 0.4344182014465332),
 ('really', 0.43325215578079224)]

In [25]:
model_w2v.wv.most_similar('one')

[("n't", 0.49487966299057007),
 ('two', 0.47774815559387207),
 ("'s", 0.46667248010635376),
 ('way', 0.4609620273113251),
 ('even', 0.457366943359375),
 ('think', 0.4486578106880188),
 ('article', 0.4449920356273651),
 ('thing', 0.4265356659889221),
 ('would', 0.419446736574173),
 ('every', 0.41300418972969055)]

In [26]:
bool_save_model = False

if bool_save_model:
    model_w2v.wv.save_word2vec_format('w2v_df_t2_clnd_sample.bin', binary = True)

In [27]:
model_w2v_vectors = model_w2v.wv # getting keyed vectors from trained model

In [128]:
# building text vectors
def tf(words_from_text, word):
    return words_from_text.count(word) / len(words_from_text)


def idf(corpus, word): # basic formula, not sklearn modification
    return np.log(len(corpus) / (np.sum(pd.Series(corpus).apply(lambda l: word in l).sum()) + 1)) # ineffective actually


def tf_idf(corpus, words_from_text, word):
    return tf(words_from_text, word) * idf(corpus, word)


def form_text_vector(words_from_text, w2v_model_keyed_vectors, num_dim, weightened = False, corpus = None):
    text_vectorized = np.zeros(num_dim)
    for word in words_from_text:
        try:
            q = tf_idf(corpus, words_from_text, word) if weightened else 1
            v = w2v_model_keyed_vectors.get_vector(word)
            s = ne.evaluate('q * v')
            text_vectorized = ne.evaluate('text_vectorized + s') 
        except KeyError:
            continue
    return text_vectorized


def form_corpus_matrix(corpus, w2v_model_keyed_vectors, num_dim, weightened = False):
    corpus_len = len(corpus)
    corpus_vectorized = np.empty((corpus_len, num_dim))
    for j in range(corpus_len):
        corpus_vectorized[j] = form_text_vector(
            corpus[j], w2v_model_keyed_vectors, num_dim, weightened, (corpus if weightened else None)
        )
    return corpus_vectorized

In [31]:
text_categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'cleaned']

temp = df[[text_categories[4], text_categories[-1]]]

temp_n = temp[~df[text_categories[:-1]].any(axis = 'columns')]
temp_i = temp[df.insult != 0]

insulting_and_neutral = temp_i.append(temp_n).reset_index(drop = True)
insulting_and_neutral.columns = ['label', 'texts']

del temp, temp_n, temp_i

print(
    insulting_and_neutral.head(),
    insulting_and_neutral.tail(),
    sep = '\n\n'
)

   label                                              texts
0      1           ['cocksucker', 'piss', 'around', 'work']
1      1  ['gay', 'antisemmitian', 'archangel', 'white',...
2      1                ['fuck', 'filthy', 'mother', 'dry']
3      1  ['stupid', 'peace', 'shit', 'stop', 'deleting'...
4      1  ['=tony', 'sidaway', 'obviously', 'fistfuckee'...

        label                                              texts
151218      0  ['``', 'second', 'time', 'asking', 'view', 'co...
151219      0  ['ashamed', 'horrible', 'thing', 'put', 'talk'...
151220      0  ['spitzer', 'umm', 'actual', 'article', 'prost...
151221      0  ['look', 'like', 'wa', 'actually', 'put', 'spe...
151222      0  ['``', '...', 'really', "n't", 'think', 'under...


In [32]:
P = 0.25

X_train_t, X_test_t, Y_train, Y_test = train_test_split(
    insulting_and_neutral['texts'], insulting_and_neutral['label'],
    test_size = P,
    random_state = 1
)

X_train_t = [literal_eval(s) for s in X_train_t.reset_index(drop = True)]
X_test_t = [literal_eval(s) for s in X_test_t.reset_index(drop = True)]

Y_train = Y_train.reset_index(drop = True)
Y_test = Y_test.reset_index(drop = True)

X_train = form_corpus_matrix(X_train_t, model_w2v_vectors, n_dimensions)
X_test = form_corpus_matrix(X_test_t, model_w2v_vectors, n_dimensions)

In [33]:
X_train

array([[-12.35104467,  -8.17274323, -18.59831111, ...,  36.7983124 ,
         12.52592548,  21.1039791 ],
       [ -2.81955897,  -9.69962908,   1.88148963, ...,  13.642529  ,
        -10.04428923,   8.03920545],
       [ -9.11592402, -28.60746697,  35.92750233, ...,  30.51325954,
          3.75637635,  13.571417  ],
       ...,
       [ -0.21584964,   1.16181156,   0.79537393, ...,   2.31340441,
          3.00284505,   1.15497546],
       [ -6.83088657,  -2.82277148,   0.39542336, ...,   6.32051712,
         -3.15185402,   0.97580142],
       [ -1.00793014, -11.55683097,  -6.5866206 , ...,  10.05888395,
        -15.74624236,  -0.84744722]])

In [34]:
X_test

array([[ -7.78701475,  -2.72190091,   6.90805295, ...,   3.40705891,
         -0.07042353,  -2.67172024],
       [ -1.9286124 ,  -7.5543409 ,  -2.27088131, ...,  11.60221306,
         -5.56597029,  18.59090474],
       [ -8.52430123,  -9.59827719,   9.66561522, ...,  21.28244998,
         -5.42605857,  21.43395831],
       ...,
       [ -4.06618702,  -4.77386539,   4.74683376, ...,   5.4642067 ,
         -4.91882995,   6.47815394],
       [-14.36065287, -11.67583142,   1.646921  , ...,  24.40385153,
         -6.56671622,  26.26983546],
       [  2.70304674,   1.84420886,   1.20486937, ...,   3.22635186,
         -1.07491895,  -0.12592049]])

In [132]:
def basic_report(y_test, y_prediction):
    confusion_matr = confusion_matrix(y_test, y_prediction)
    print("CONFUSION MATRIX:\n{matr}".format(matr=confusion_matr))
    accuracy_of_model = accuracy_score(y_test, y_prediction)
    print("ACCURACY:\n{acc}".format(acc = accuracy_of_model))
    sklearn_report = classification_report(y_test, y_prediction)
    print("TABLE:\n{tab}".format(tab=sklearn_report))

# too slow, rewrite later (1)
def quick_init_and_train_word_cls_model_no_args_w2v(
    x_train_t, x_test_t, y_train, y_test, word_model, n_dim, cls_model, weightened = False):
    print('DATA PREPAIRING START')
    x_train = form_corpus_matrix(x_train_t, word_model, n_dim, weightened)
    x_test = form_corpus_matrix(x_test_t, word_model, n_dim, weightened)
    
    print('MODEL TRAINING START')
    cls_m = cls_model()
    cls_m.fit(x_train, y_train)
    
    y_prediction = cls_m.predict(x_test)
    basic_report(y_test, y_prediction)
    
    return cls_m

In [144]:
from gensim.models import TfidfModel
#X_train_t
model = TfidfModel([[[1,2],[1,2]]])
X_train_t

[['``',
  'history/infor',
  'link',
  'found',
  'kinda',
  'spammy',
  'someone',
  'went',
  'school',
  'silverdale',
  'idea',
  '``',
  "''",
  'maple',
  'fall',
  "''",
  "''",
  "'re",
  'talking',
  'maybe',
  'silver',
  'creek',
  'far',
  'east',
  'end',
  'silverdale',
  'road',
  "n't",
  'know',
  'seems',
  'genstar',
  "'s",
  're-branding',
  'silverhill',
  'silverhill/dale',
  'community',
  'fought',
  'hard',
  'i.e',
  'wiping',
  'name',
  'silverhill',
  'map',
  "'s",
  'silverdale',
  "'s",
  '``',
  "''",
  'twin',
  "''",
  "''",
  'community',
  'share',
  'family',
  'roots/connections',
  "'s",
  'citation',
  'silverdale',
  'cpr',
  'stop',
  'register',
  'heritage',
  'place',
  'wa',
  'surprised',
  'see',
  'cat',
  'removed',
  're-added',
  'heritage',
  'ref',
  'easy',
  'find',
  'dispute',
  'netlibrary.net',
  'bio',
  'proof',
  'silverdale',
  'robbery',
  'wa',
  "'ve",
  'never',
  'seen',
  'before.',
  '``'],
 ['edits',
  'athena',


In [105]:
# include classifier: RF
random_forest_cls = quick_init_and_train_word_cls_model_no_args_w2v(
    X_train_t, X_test_t, Y_train, Y_test, model_w2v_vectors, n_dimensions, RandomForestClassifier, False
)

CONFUSION MATRIX:
[[35771    44]
 [ 1002   989]]
ACCURACY:
0.9723324340051843
TABLE:
              precision    recall  f1-score   support

           0       0.97      1.00      0.99     35815
           1       0.96      0.50      0.65      1991

    accuracy                           0.97     37806
   macro avg       0.97      0.75      0.82     37806
weighted avg       0.97      0.97      0.97     37806



In [106]:
# include classifier: LR
logit_cls = quick_init_and_train_word_cls_model_no_args_w2v(
    X_train_t, X_test_t, Y_train, Y_test, model_w2v_vectors, n_dimensions, LogisticRegression
)

CONFUSION MATRIX:
[[35632   183]
 [  618  1373]]
ACCURACY:
0.9788128868433582
TABLE:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     35815
           1       0.88      0.69      0.77      1991

    accuracy                           0.98     37806
   macro avg       0.93      0.84      0.88     37806
weighted avg       0.98      0.98      0.98     37806



c:\users\laplace-transform\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


#### The second part of the task is: 

1. While performing a step 2 for text vectorization, for each word add its vector with tf-idf weight -> weighted average. 
2. Perform a same text classification task as it was required above. 
3. Calculate the metrics, compare with a vectorization approach without weightning. 

In [133]:
random_forest_cls = quick_init_and_train_word_cls_model_no_args_w2v(
    X_train_t, X_test_t, Y_train, Y_test, model_w2v_vectors, n_dimensions, RandomForestClassifier, True
) # (1)

DATA PREPAIRING START


KeyboardInterrupt: 

#### The third part of the task is: 

1. Use a pre-trained W2V model for obtaining a word vectors for each of the tokens in your dataset, create text vectors WITHOUT weightning. 
2. Train text classification model.
3. Calculate the metrics.

In [107]:
# include classifier: RF
random_forest_cls = quick_init_and_train_word_cls_model_no_args_w2v(
    X_train_t, X_test_t, Y_train, Y_test, model, n_dimensions, RandomForestClassifier
)

CONFUSION MATRIX:
[[35794    21]
 [ 1233   758]]
ACCURACY:
0.9668306617997143
TABLE:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     35815
           1       0.97      0.38      0.55      1991

    accuracy                           0.97     37806
   macro avg       0.97      0.69      0.77     37806
weighted avg       0.97      0.97      0.96     37806



#### The fourth part of the task is: 

1. Use a pre-trained W2V model for obtaining a word vectors for each of the tokens in your dataset, create text vectors WITH tf-idf weightning. 
2. Train a text classification model. 
3. Calculate the metrics. 

In [ ]:
random_forest_cls = quick_init_and_train_word_cls_model_no_args_w2v(
    X_train_t, X_test_t, Y_train, Y_test, model, n_dimensions, RandomForestClassifier, True
)

### Visualizations part 

Use dimentionality reduction methods such as t-SNE or PCA to make your 300 dim vectors available for 2D plotting. 

Select top (10-20) words for each cathegory BY TF-IDF SCORE, not counts!!! 

Plot on the ONE plot all of this words but colors must be different for top-words for obscene cathegory, clean, toxic, etc... 

See, if words from one cathegory are closer to each other than to others. 
Or you observe ~2 clusters: all of the toxic words, clean words.  
Explain what you see and why. 


In [ ]:
# as always, using PCA
model_pca = PCA(n_components = 5)

### Additional part: 

1. Find a pre-trained FastText vectors, understand it's difference from W2V vectors. 
2. Vectorize all of your texts using FT model, perform a text classification, calculate the metrics, compare with W2V approach. 

Or/And you can:

1. Train your own FT model and make the same. 
2. Compare it with previous approaches.

In [97]:
model_ft = FastText(sentences=df_sample_cleaned_list, 
                    size=300,
                    min_count=5,
                    window=5)

In [98]:
# model training
number_of_iterations = 50

model_ft.train(sentences=df_sample_cleaned_list, 
            total_examples=model_ft.corpus_count,
            epochs=number_of_iterations
           )

### Conclusions: 

Please, provide a clear table or dataframe with all of the metrics for all of the trained/used models available.   

Compare them to each other.   

Make conclusions which one from your models worked better for this particular task.   
BE CAREFUL: Having a better model performance on this particular task does not matter that this model is better than others in GENERAL. You need to make your own conclusions about this particular model applied to this particular task. Please, think and understand WHY.   
Write your thoughts down below: 



In [ ]:
### Your conclusions here.

In [ ]:
### Your thoughts about the last question here. 

In [74]:
model_w2v_vectors.get_vector('daniel').shape

(300,)